In [60]:
import pandas as pd
import os

os.chdir("/Users/rasmus/Documents/Virksomhed/aalegraes")
inFile = "vanda_vanda-ue-33.csv"
outFile = "aalegraes_chunk.csv"

# Læs et udsnit, fx 500 rækker
chunk = pd.read_csv(
    inFile,
    sep=";", 
    decimal=",", 
    nrows=500,
    low_memory=False
)

# Gem til en ny fil
chunk.to_csv(outFile, sep=";", index=False)

print("Gemte chunk:", outFile)


Gemte chunk: aalegraes_chunk.csv


In [61]:
import pandas as pd
import os

# Stier til input/output
inFolder = "/Users/rasmus/Documents/Virksomhed/aalegraes"
inFile = os.path.join(inFolder, "vanda_vanda-ue-33.csv")
outFile = os.path.join(inFolder, "aalegraes.parquet")

print("Indlæser CSV...")

'''
# læs et udsnit
df = pd.read_csv(
    inFile,
    sep=";", 
    decimal=",", 
    nrows=100_000,
    low_memory=False
    encoding="utf-8-sig"   # fjerner evt. BOM i header
)
'''

# Læs hele CSV
df = pd.read_csv(
    inFile,
    sep=";", 
    decimal=",", 
    low_memory=False,
    encoding="utf-8-sig"   # fjerner evt. BOM i header
)

print("Rækker indlæst:", len(df))

# Konverter dato (dd.mm.yyyy HH.MM.SS +zz:zz)
df["Dato"] = pd.to_datetime(
    df["Dato"],
    format="%d.%m.%Y %H.%M.%S %z",
    utc=True,
    errors="coerce"
)

print("Antal gyldige datoer:", df["Dato"].notna().sum())
print("Antal NaT (fejl):", df["Dato"].isna().sum())

# Numeriske kolonner
for col in ["Dybde", "Længde", "Dækningsgrad"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Tekstfelter som kategorier (pladsbesparende)
cat_cols = [
    "Observations gruppe","Observation","Stedtekst","Referencer","Kommune","Region",
    "Medie","Vandområde","Undersøgelsestype","Teknisk anvisning","Dataejer",
    "Transekt navn","T-stykke"
]
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].astype("category")

# Gem til Parquet
df.to_parquet(outFile, engine="pyarrow", index=False)

print("Gemte Parquet:", outFile)
#print("Eksempel på data:")
#print(df.head())
#print("Datatyper:")
#print(df.dtypes)


Indlæser CSV...
Rækker indlæst: 3884527
Antal gyldige datoer: 3884527
Antal NaT (fejl): 0
Gemte Parquet: /Users/rasmus/Documents/Virksomhed/aalegraes/aalegraes.parquet
Eksempel på data:
  Stedtype    StedID              Stedtekst             Referencer  GeoZone  \
0  Station  93720104  Tr. 17, Nibe Bredning  Tr. 17, Nibe Bredning       32   
1  Station  94410079    ÅB 0007, Vosnæsgård    ÅB 0007, Vosnæsgård       32   
2  Station  96540058               Ærø N-NW               Ærø N-NW       32   
3  Station  96260230           0201101 Askø           0201101 Askø       32   
4  Station  93760076        Tr. 09, Sennels        Tr. 09, Sennels       32   

     x-koordinat   y-koordinat           Kommune           Region  Medie  ...  \
0  537551.532981  6.317611e+06  Uden for kommune  Uden for region  Marin  ...   
1  585857.898685  6.237701e+06  Uden for kommune  Uden for region  Marin  ...   
2  581826.527883  6.089216e+06  Uden for kommune  Uden for region  Marin  ...   
3  658642.24762

In [65]:
# Sanity tjek
import pandas as pd

# Indlæs parquet
df = pd.read_parquet("aalegraes.parquet")

print("Shape:", df.shape)

# Datatyper
print("\nDatatyper:")
print(df.dtypes)

# Første par rækker
print("\nHead:")
print(df.head())

# --- Tjek Dato ---
print("\nDato dtype:", df["Dato"].dtype)
print("Antal NaT i Dato:", df["Dato"].isna().sum())

# De første 10 datoer
print("\nFørste 10 datoer:")
print(df["Dato"].head(10).to_list())

# Et tilfældigt udsnit på 20 datoer
print("\nTilfældigt udsnit af 20 datoer:")
print(df["Dato"].dropna().sample(20, random_state=42).to_list())

# --- Tjek numeriske kolonner ---
for col in ["Dybde", "Længde", "Dækningsgrad"]:
    if col in df.columns:
        print(f"\n{col}:")
        print("  dtype:", df[col].dtype)
        print("  min:", df[col].min())
        print("  max:", df[col].max())
        print("  antal NaN:", df[col].isna().sum())


Shape: (3884527, 38)

Datatyper:
Stedtype                                    object
StedID                                       int64
Stedtekst                                 category
Referencer                                category
GeoZone                                      int64
x-koordinat                                float64
y-koordinat                                float64
Kommune                                   category
Region                                    category
Medie                                     category
Vandområde                                category
Dato                           datetime64[ns, UTC]
Måling nr                                    int64
Undersøgelsestype                         category
Teknisk anvisning                         category
Dataejer                                  category
Link                                        object
Transekt nummer                              int64
Transekt navn                             categor

In [66]:
import pandas as pd

os.chdir("/Users/rasmus/Documents/Virksomhed/aalegraes")

# læs hele datasættet (allerede konverteret til Parquet)
df = pd.read_parquet("aalegraes.parquet")

print("Antal rækker:", len(df))
print("Kolonner:", df.columns.tolist())

Antal rækker: 3884527
Kolonner: ['Stedtype', 'StedID', 'Stedtekst', 'Referencer', 'GeoZone', 'x-koordinat', 'y-koordinat', 'Kommune', 'Region', 'Medie', 'Vandområde', 'Dato', 'Måling nr', 'Undersøgelsestype', 'Teknisk anvisning', 'Dataejer', 'Link', 'Transekt nummer', 'Transekt navn', 'Transekt-start, GeoZone', 'Transekt-start, x-koordinat', 'Transekt-start, y-koordinat', 'Transekt-slut, GeoZone', 'Transekt-slut, x-koordinat', 'Transekt-slut, y-koordinat', 'Målested navn', 'Målested, GeoZone', 'Målested, x-koordinat', 'Målested, y-koordinat', 'T-stykke', 'Længde', 'Dybde', 'Observations gruppe', 'Observation', 'Dækningsgrad', 'Hovedudbredelse', 'Maks dybdegrænse', 'Kvalitetsmærke']


In [69]:
import pandas as pd

# --- 2a. Minimum antal undersøgelser pr. StedID ---
counts = df.groupby("StedID")["Dato"].nunique().reset_index(name="n_undersøgelser")

print("Alle steder i datasættet:", df["StedID"].nunique())
print("Steder med >=10 undersøgelser:", (counts["n_undersøgelser"] >= 10).sum())

# vælg kun relevante steder
relevante = counts[counts["n_undersøgelser"] >= 10]["StedID"]
df_filt = df[df["StedID"].isin(relevante)]

# --- 2b. Seneste undersøgelse indenfor X år ---
cutoff = (pd.Timestamp.today(tz="UTC") - pd.DateOffset(years=6))
df_filt = df_filt.groupby("StedID").filter(lambda g: (g["Dato"] >= cutoff).any())

print(f"\nEfter tidsfilter (>= én undersøgelse siden {cutoff.date()}):")
print("Antal steder:", df_filt["StedID"].nunique())

# --- 2c. Geografisk afgrænsning (bounding box) ---
print("\nKoordinat-udstrækning i hele datasættet:")
print("x:", df["x-koordinat"].min(), "→", df["x-koordinat"].max())
print("y:", df["y-koordinat"].min(), "→", df["y-koordinat"].max())

xmin, xmax = 600000, 800000
ymin, ymax = 6100000, 6200000

df_filt = df_filt[
    df_filt["x-koordinat"].between(xmin, xmax) &
    df_filt["y-koordinat"].between(ymin, ymax)
]

print(f"\nEfter geografisk filter (x={xmin}-{xmax}, y={ymin}-{ymax}):")
print("Antal steder:", df_filt["StedID"].nunique())
print("Antal observationer:", len(df_filt))

# --- 2d. Oversigt over de filtrerede steder ---
sted_counts = df_filt.groupby("StedID")["Dato"].nunique().reset_index(name="antal_undersøgelser")
print("\nListe over filtrerede StedID og antal undersøgelser:")
print(sted_counts.sort_values("antal_undersøgelser", ascending=False).head(20))  # top 20


Alle steder i datasættet: 2002
Steder med >=10 undersøgelser: 500

Efter tidsfilter (>= én undersøgelse siden 2019-09-25):
Antal steder: 342

Koordinat-udstrækning i hele datasættet:
x: 445438.2674810079 → 882971.1918629045
y: 6047484.57899734 → 6381365.023262053

Efter geografisk filter (x=600000-800000, y=6100000-6200000):
Antal steder: 91
Antal observationer: 656588

Liste over filtrerede StedID og antal undersøgelser:
      StedID  antal_undersøgelser
74  97220010                   53
73  97210033                   53
69  97120088                   51
87  99350071                   41
8   93220079                   38
71  97130005                   35
0   93210066                   32
70  97120092                   32
56  96220417                   28
6   93220073                   28
75  97220011                   27
20  94120057                   27
12  93220107                   27
1   93210069                   27
76  97230018                   26
9   93220087                  

In [70]:
stations = []

for (stedid, stednavn), g in df_filt.groupby(["StedID", "Stedtekst"]):
    x = float(g["x-koordinat"].iloc[0])
    y = float(g["y-koordinat"].iloc[0])
    
    # H (hovedudbredelse)
    h = g[g["Hovedudbredelse"] == "H"].groupby(g["Dato"].dt.year)["Dybde"].agg(["mean","std"]).reset_index()
    series_h = [{"year": int(r["Dato"]), "mean": float(r["mean"]), "std": (None if pd.isna(r["std"]) else float(r["std"]))} for _, r in h.iterrows()]
    
    # M (maksimal dybdegrænse)
    m = g[g["Maks dybdegrænse"] == "M"].groupby(g["Dato"].dt.year)["Dybde"].agg(["mean","std"]).reset_index()
    series_m = [{"year": int(r["Dato"]), "mean": float(r["mean"]), "std": (None if pd.isna(r["std"]) else float(r["std"]))} for _, r in m.iterrows()]
    
    stations.append({
        "stedid": int(stedid),
        "stednavn": stednavn,
        "easting": x,
        "northing": y,
        "crs": "EPSG:25832",
        "seriesH": series_h,
        "seriesM": series_m
    })

print("Stations klar:", len(stations))


/var/folders/73/gx_39dj57n5fl666wldpg4_00000gn/T/ipykernel_40303/2223745182.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (stedid, stednavn), g in df_filt.groupby(["StedID", "Stedtekst"]):


Stations klar: 91


In [73]:
import json

with open("aalegraes_stations.json", "w", encoding="utf-8") as f:
    json.dump({"stations": stations}, f, ensure_ascii=False, indent=2)

print("Gemte aalegraes_stations.json")


Gemte aalegraes_stations.json
